In [56]:
import pandas as pd
import numpy as np
import re
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import cv2
import os,glob

# Writing the ocr strings to seperate text files

In [63]:
'''path = '../data/img'
tempPath ="../data/text"
for imageName in os.listdir(path):
    inputPath = os.path.join(path, imageName)
    img = cv2.imread(inputPath)
    img2 = cv2.resize(img ,None,fx=1.5,fy=1.5)
    gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    adaptive = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,87,11)

    text = pytesseract.image_to_string(adaptive)

    #imagePath = imagePath[0:-4]

    fullTempPath = os.path.join(tempPath, imageName+".txt")

    # saving the  text for every image in a separate .txt file
    file1 = open(fullTempPath, "w")
    file1.write(text)
    file1.close() '''

# Reading the text files and create the features dataframe

In [106]:
receipt_ocr = {}
date_lst=[]
time_lst=[]
item_count =[]

In [107]:
item_count

[]

In [115]:
txtFilePath ="../data/text"
for fileName in os.listdir(txtFilePath):
    FilePath = os.path.join(txtFilePath, fileName)
    f=open(FilePath,'r')
    k=''
    for line in f:
        l=line
        k+=l
# date
    date_pattern = r'(0[1-9]|[12][0-9]|3[01])([/]|[-])(0[1-9]|1[012])([/]|[-])(19|20)\d\d'
    date = re.search(date_pattern, k)
    if type(date) != type(None):
        date =  date.group()
    date_lst.append(date)
# time
    time_pattern = r'(\d\d|\d)[:](\d\d)[:](\d\d)'
    time = re.search(time_pattern, k)
    if type(time) != type(None):
        time =  time.group()
    time_lst.append(time)

#quantity

    splits = k.splitlines()
    for line in splits:
        if re.search(r'Item Count',line):
            lines_with_item_count=line
            seq = lines_with_item_count.split(" ")
            print(seq)
            for charactors in seq:
                #print(charactors)
                try:
                    char = int(charactors) 
                    total_item_count = char
                                        
                except :
                    continue
            item_count.append(total_item_count)
    else:
            item_count.append(0)
         

['Item', 'Count:', '2']
['Main', 'Item', 'Count']
['Item', 'Count', '5']
['Item', 'Count:', '4', 'Item', 'Qty:', '5.00']
['Item', 'Count', '2']
['Item', 'Count', '1°', '©', 'Change', 'Amt', '—-—~', '44,10']
['Item', 'Count', '1', 'Change', 'Amt', '4,10']
['Item', 'Count', '3']
['Item', 'Count', '2']
['Item', 'Count', '2']
['Item', 'Count', ';', '2']
['Item', 'Count', '32']
['Item', 'Count', '4', 'Chavde', 'Amt', '7.56']
['Item', 'Count', '5']
['Item', 'Count', '6']
['Item', 'Count', '2']
['Item', 'Count']
['Item', 'Count']
['Item', 'Count']
['Item', 'Count', ':', '50']
['Item', 'Count']
['Item', 'Count', '7', 'sina', 'Amt', '0.05']
['Item', 'Count', ':', '“', 'the', 'Change', 'Amt', 'B2,', '45']
['Item', 'Count', '2', 'Change', 'Amt', '46.10']
['Item', 'Count', '7', 'Change', 'Amt', '0.10']
['Item', 'Count', '4', 'Change', 'Amt', '23.90']
['Item', 'Count', ':', '50']
['Item', 'Count', '12', 'Caange', 'Amt', '22.09']
['Item', 'Count', '3', 'Change.', 'Amt', '11.40']


In [116]:
receipt_ocr['date'] = date_lst
receipt_ocr['time'] = time_lst
receipt_ocr['item count'] = item_count

In [117]:
len(item_count)

30630

In [68]:
df_features = pd.DataFrame(receipt_ocr)

ValueError: arrays must all be same length

In [42]:
df_features

,date,time
0,25/12/2018,8:13:39
1,19/10/2016,20:49:59
2,None,None
3,25/12/2018,None
4,None,None
...,...,...
596,05-06-2018,20:23:14
597,22-05-2018,19:54:52
598,28-05-2018,23:28:57
599,19-05-2018,12:46:58


In [9]:
import pandas as pd
results = pd.read_json (r'results-all.json')
print (results)

    file_name   total
0    000.json    9.00
1    001.json   60.30
2    002.json   33.90
3    003.json   80.90
4    004.json   30.90
..        ...     ...
596  596.json   52.00
597  597.json   71.00
598  598.json  112.35
599  599.json   42.40
600  600.json    4.70

[601 rows x 2 columns]


In [10]:
# check missing values

num_of_null_columns = len([col for col in df_features.columns if sum(df_features[col].isnull())])
print(num_of_null_columns)

2


In [11]:
# check for duplications

df_features_len_before = len(df_features)
df_features.drop_duplicates()
df_features_len_after = len(df_features)
print(df_features_len_before-df_features_len_after)

0


# Split the data into training and validation

In [12]:
X = df_features.copy()
y = results['total']

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.8,random_state=0)

In [50]:
from sklearn.impute import SimpleImputer
import pandas as pd
# Imputation
my_imputer = SimpleImputer(strategy='constant',fill_value='18/09/2018')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

In [51]:
imputed_X_train

,date,time
0,None,None
1,None,None
2,None,None
3,02/01/2018,09:28:00
4,14-03-2018,10:22:59
...,...,...
475,None,03:17:50
476,18/01/2018,10:27:03
477,20/10/2017,None
478,None,None


In [33]:

num_of_null_columns2 = len([col for col in imputed_X_train.columns if sum(imputed_X_train[col].isnull())])
print(num_of_null_columns2)

2
